In [28]:
from imdb import IMDb
import time
import pickle

In [61]:
ia = IMDb()
movie1 = ia.get_movie('9219827')
print movie1

2017-03-31 14:00:34,779 WARNING [imdbpy.parser.http.urlopener] /Users/hung-yiwu/anaconda/lib/python2.7/site-packages/IMDbPY-5.1-py2.7-macosx-10.6-x86_64.egg/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt9219827/combined: Not Found (headers: Date: Fri, 31 Mar 2017 18:00:34 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYjTxsuboGqyOZ3YXjGMnZ19QqZwWDrPpq_hNwdRJ3XCv__7hKohNo5PYFQnIm_6EKA4gxcRZ3OTY0GEuvH_IsjUUZ7QLzTYxkcb8Xzxrwpbuk;path=/;domain=.imdb.com
Set-Cookie: uu=BCYsXSKIejU6NbtUhBukT7F7pguZwWDrPpq_hNwdRJ3XCv8sPjlhWc8siYjyjSJcfDSvA-aQ0mXUP3YVt1nI3739bewKBYuLb7iD47Aobmd8lvXVAJJnPI6eWjn5LGxY_aut6ylOm8Z42FzO-U7Ff0K3s7eA5VJwn0CrPGthEz0nFHjWuuXIc1r_fxKerzv466FS1cQzm3exxU--eRVpP13KQDYfO_qEtxgNt6xB1Riz3z0cQT2LF5FZ-oVZw7efOO3xi0sFm0tUR4PmoO8QkTHYnVGYBQBHCzwZyZcSPY_qYEOGfTIEJuCHKlhmQhmH_pltW0KFJ7lA4-r_v8KGYqrePg;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.c

In [47]:
t0 = time.time()
movie_list = []
for number in range(0, 20):
    movie_id = str(number).zfill(7)
    current_movie = ia.get_movie(movie_id)
    if len(current_movie) is not 0:
        movie_list.append(current_movie)
t1 = time.time()
print (t1-t0)/20

2017-03-31 13:38:48,491 WARNING [imdbpy.parser.http.urlopener] /Users/hung-yiwu/anaconda/lib/python2.7/site-packages/IMDbPY-5.1-py2.7-macosx-10.6-x86_64.egg/imdb/parser/http/__init__.py:248: 404 code returned for //akas.imdb.com/title/tt0000000/combined: Not Found (headers: Date: Fri, 31 Mar 2017 17:38:47 GMT
Server: HTTPDaemon
X-Frame-Options: SAMEORIGIN
Cache-Control: private
Content-Type: text/html; charset=iso-8859-1
Set-Cookie: cache=BCYhfKd_-3fY5cXKj9YFa6vYgJNdTWGEDk1pmtAyJZSEXhqXstfvtFby5S_8gCNgrswYOB0eetxJpSB7FCHADKp_omavDD8kdBMtLdZNKB0isqg;path=/;domain=.imdb.com
Set-Cookie: uu=BCYomu4Qnt4TQEnXgRBlzPKhj5RdTWGEDk1pmtAyJZSEXho74QO0AlGRlEIdQvuFh-TEMcWahyjrXfbxgO_aDP90fIZesFlT7_hdJd3VAOkRW48YO--eYIWJrLp_1zLEADScHF_vxH1McA-2BK_hioj4SCA1_q3Rbq9-YgGlO95OdcdX6AOIHDjUwCloR19enc6oQp1NPtMwY3z_ODnWbVOFr4mW6D29L1Kwur5CN7lvFXBrKBnabz43Dx1HKgheu_bfeDYm1s5vqKGsNFc3n1y2ET65cBvemF4t1PH5PwiDzYj-oVswU_ZL_5i7ZSe7mmumHs1WQMWhz_WdtKedzcJESg;expires=Thu, 30 Dec 2037 00:00:00 GMT;path=/;domain=.imdb.c

1.19980905056


In [44]:
with open("test.txt", "wb") as myfile:
    pickle.dump(movie_list, myfile)

In [56]:
with open("test.txt", "rb") as myfile:
    b = pickle.load(myfile)
for movie in b:
    print movie['year']

1894
1892
1892
1892
1893
1894
1894
1894
1894
1895
1895
1896
1895
1895
1894
1895
1895
1895
1898
